# Dynamics lineshapes

In [ ]:
import sympy as sp
from ampform.kinematics.phasespace import Kallen

from polarimetry.dynamics import (
    BlattWeisskopf,
    BreitWignerMinL,
    BuggBreitWigner,
    EnergyDependentWidth,
    FlattéSWave,
    P,
    Q,
)
from polarimetry.io import display_doit, display_latex

In [ ]:
z = sp.Symbol("z", positive=True)
L = sp.Symbol("L", integer=True, nonnegative=True)
display_doit(BlattWeisskopf(z, L))

In [ ]:
x, y, z = sp.symbols("x:z")
display_doit(Kallen(x, y, z))

In [ ]:
s, m0, mi, mj, mk = sp.symbols("s m0 m_i:k", nonnegative=True)
display_doit(P(s, mi, mj))
display_doit(Q(s, m0, mk))

In [ ]:
R = sp.Symbol("R")
l_R = sp.Symbol("l_R", integer=True, positive=True)
m, Γ0, m1, m2 = sp.symbols("m Γ0 m1 m2", nonnegative=True)
display_doit(EnergyDependentWidth(s, m, Γ0, m1, m2, l_R, R))

## Relativistic Breit-Wigner

In [ ]:
m_top, m_spec = sp.symbols(R"m_\mathrm{top} m_\mathrm{spectator}")
R_dec, R_prod = sp.symbols(R"R_\mathrm{res} R_{\Lambda_c}")
l_Λc = sp.Symbol(R"l_{\Lambda_c}", integer=True, positive=True)
display_doit(
    BreitWignerMinL(s, m_top, m_spec, m, Γ0, m1, m2, l_R, l_Λc, R_dec, R_prod)
)

## Bugg Breit-Wigner

In [ ]:
mKπ, m0, Γ0, mK, mπ, γ = sp.symbols(R"m_{K\pi} m0 Gamma0 m_K m_pi gamma")
bugg = BuggBreitWigner(mKπ**2, m0, Γ0, mK, mπ, γ)
q = P(mKπ**2, mK, mπ)
s_A = sp.Symbol("s_A")
definitions = {
    s_A: mK**2 - mπ**2 / 2,
    q: q.evaluate(),
}
display_latex({bugg: bugg.evaluate().subs({v: k for k, v in definitions.items()})})
display_latex(definitions)

One of the models uses a Bugg Breit-Wigner with an exponential factor:

In [ ]:
q = Q(s, m0, m1)
alpha = sp.Symbol("alpha")
bugg * sp.exp(-alpha * q**2)

## Flatté for S-waves

In [ ]:
Γ1, Γ2, m1_1, m2_1, m1_2, m2_2 = sp.symbols("Gamma1 Gamma2 m1_1 m2_1 m1_2 m2_2")
display_doit(FlattéSWave(s, m, (Γ1, Γ2), (m1_1, m2_1), (m1_2, m2_2)))